In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:


train = pd.read_csv('/kaggle/input/ml-league-supervised-learning-competition/train.csv')
test = pd.read_csv('/kaggle/input/ml-league-supervised-learning-competition/test.csv')
sample_submission = pd.read_csv('/kaggle/input/ml-league-supervised-learning-competition/sample_submission.csv')

# Show a quick preview
train.head()


# FIRT SUBMISSION USING XGBOOST 

In [ ]:

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor


In [ ]:
# Droppping unused columns
drop_cols = ['id', 'Row#']  
X = train.drop(columns=drop_cols + ['yield'])
y = train['yield']

X_test = test.drop(columns=drop_cols)

# Filling the  missing values 
X.fillna(X.mean(), inplace=True)
X_test.fillna(X_test.mean(), inplace=True)


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
model = XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)

# Validation predictions
val_preds = model.predict(X_val)
val_mae = mean_absolute_error(y_val, val_preds)
print(f"Validation MAE: {val_mae:.4f}")


In [ ]:

test_preds = model.predict(X_test)

submission = pd.DataFrame({
    'id': test['id'],
    'yield': test_preds})

submission.to_csv('submission.csv', index=False)


PANDAS PROFILIG DIDNT WORK THREE TIMES, KAGGLE BECAME UNREPONSIVE SO DID EDA MANUALLY.

# EDA PIPELINE

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import warnings
warnings.filterwarnings("ignore")



# Basic Info + Missing Values

In [ ]:
print(" \nShape:", train.shape)
print("\n Data Types:\n", train.dtypes)
print("\n Missing Values:\n", train.isnull().sum().sort_values(ascending=False))
print("\n Descriptive Stats:\n", train.describe())


#  Distribution & Skewness of Features

In [ ]:
numeric_features = train.select_dtypes(include=[np.number]).columns.tolist()
numeric_features.remove("yield")  # Excluding target for now

# Skewness
skewed = train[numeric_features].skew().sort_values(ascending=False)
print("\n🧬 Feature Skewness:\n", skewed)

# distributions
for col in numeric_features:
    plt.figure(figsize=(6, 4))
    sns.histplot(train[col], kde=True, bins=30)
    plt.title(f"Distribution of {col}")
    plt.show()


# correlation map

In [ ]:
plt.figure(figsize=(14, 10))
corr = train.corr()
sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm", square=True)
plt.title(" Correlation Matrix")
plt.show()


In [ ]:
target_corr = corr["yield"].sort_values(ascending=False)
print(" Features Most Correlated with 'yield':\n", target_corr)


# outliers

In [ ]:
for col in numeric_features:
    plt.figure(figsize=(6, 4))
    sns.boxplot(x=train[col])
    plt.title(f"Boxplot of {col}")
    plt.show()


#  Relationships to Target

In [ ]:
for col in numeric_features:
    plt.figure(figsize=(6, 4))
    sns.scatterplot(x=train[col], y=train['yield'])
    plt.title(f"{col} vs Yield")
    plt.show()


# statistical test

The Shapiro–Wilk test is a statistical method used to determine whether a given dataset follows a normal distribution (also known as a Gaussian distribution). This test is commonly applied in data analysis to check one of the key assumptions of many statistical models, such as linear regression — that the residuals or inputs are normally distributed. The test works by comparing the order statistics (sorted values) of the sample to what would be expected from a normal distribution. It returns a test statistic and a p-value: if the p-value is less than a chosen significance level (typically 0.05), the null hypothesis — that the data is normally distributed — is rejected. In practice, this means that a low p-value indicates non-normal data, while a high p-value suggests the data is consistent with a normal distribution. The Shapiro–Wilk test is most effective on small to moderately sized samples (usually n < 5000).

In [ ]:
# Shapiro-Wilk test for normality
for col in numeric_features:
    stat, p = stats.shapiro(train[col].dropna())
    print(f"{col}: p={p:.4f} {'(Not normal)' if p < 0.05 else '(Probably normal)'}")


In [ ]:
from sklearn.ensemble import RandomForestRegressor

X = train.drop(columns=['id', 'Row#', 'yield'])
y = train['yield']

model = RandomForestRegressor(random_state=42)
model.fit(X, y)

importances = pd.Series(model.feature_importances_, index=X.columns)
importances.sort_values().plot(kind='barh', figsize=(10, 8), title="Feature Importance (Random Forest)")
plt.show()


In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# Define data
features = ['fruitset', 'seeds', 'fruitmass']
X =train[features]
y =train['yield']

# Train/Val Split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train LightGBM
model = lgb.LGBMRegressor(n_estimators=1000, learning_rate=0.05)
model.fit(X_train, y_train, eval_set=[(X_val, y_val)])

# Evaluate
y_pred = model.predict(X_val)
mae = mean_absolute_error(y_val, y_pred)
print(f"📉 MAE: {mae:.4f}")


In [ ]:
test = pd.read_csv("/kaggle/input/ml-league-supervised-learning-competition/test.csv")
X_test = test[['fruitset', 'seeds', 'fruitmass']]
y_test_pred = model.predict(X_test)
submission = pd.read_csv("/kaggle/input/ml-league-supervised-learning-competition/sample_submission.csv")
submission['yield'] = y_test_pred
submission.to_csv("submission2.csv", index=False)
print(" submission.csv saved successfully!")



In [ ]:
import os
os.listdir("/kaggle/working/")


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor

# Load the training data
train = pd.read_csv("/kaggle/input/ml-league-supervised-learning-competition/train.csv")

# Features and target
features = ['fruitset', 'seeds', 'fruitmass']
X = train[features]
y = train['yield']

# Train/validation split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

results = {}

# XGBoost
xgb_model = XGBRegressor(n_estimators=500, learning_rate=0.05, random_state=42)
xgb_model.fit(X_train, y_train)
xgb_preds = xgb_model.predict(X_val)
results['XGBoost'] = mean_absolute_error(y_val, xgb_preds)

# Random Forest
rf_model = RandomForestRegressor(n_estimators=300, random_state=42)
rf_model.fit(X_train, y_train)
rf_preds = rf_model.predict(X_val)
results['RandomForest'] = mean_absolute_error(y_val, rf_preds)

# HistGradientBoosting
hgb_model = HistGradientBoostingRegressor(random_state=42)
hgb_model.fit(X_train, y_train)
hgb_preds = hgb_model.predict(X_val)
results['HistGradientBoosting'] = mean_absolute_error(y_val, hgb_preds)

# KNN (with scaling)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
knn_model = KNeighborsRegressor(n_neighbors=5)
knn_model.fit(X_train_scaled, y_train)
knn_preds = knn_model.predict(X_val_scaled)
results['KNN'] = mean_absolute_error(y_val, knn_preds)

# Display all MAEs
for model, mae in results.items():
    print(f"{model} MAE: {mae:.4f}")


In [ ]:
test = pd.read_csv("/kaggle/input/ml-league-supervised-learning-competition/test.csv")
X_test = test[['fruitset', 'seeds', 'fruitmass']]
y_test_pred = hgb_model.predict(X_test)
submission = pd.read_csv("/kaggle/input/ml-league-supervised-learning-competition/sample_submission.csv")
submission['yield'] = y_test_pred
submission.to_csv("submission3.csv", index=False)
print(" submission3.csv saved successfully!")


In [ ]:
import pandas as pd
from ydata_profiling import ProfileReport

# Load your dataset
df = pd.read_csv('/kaggle/input/ml-league-supervised-learning-competition/train.csv')

# Generate the report
profile = ProfileReport(df, title="EDA Report", explorative=True,minimal = True)

# Save or display it
profile.to_file("eda_report.html")
# OR in notebook
profile.to_notebook_iframe()


# submisson 4

In [ ]:

train = pd.read_csv("/kaggle/input/ml-league-supervised-learning-competition/train.csv")
test = pd.read_csv("/kaggle/input/ml-league-supervised-learning-competition/test.csv")


features = ['fruitset', 'seeds', 'fruitmass']


for df in [train, test]:
    df['fruitset_x_mass'] = df['fruitset'] * df['fruitmass']
    df['seeds_div_mass'] = df['seeds'] / (df['fruitmass'] + 1e-6)  # avoid divide by zero
    df['fruit_density'] = df['fruitmass'] / (df['seeds'] + 1e-6)

new_features = features + ['fruitset_x_mass', 'seeds_div_mass', 'fruit_density']


In [ ]:

best_model.fit(X, y)



X_test = test[new_features]
test_preds = best_model.predict(X_test)


sample_submission = pd.read_csv("/kaggle/input/ml-league-supervised-learning-competition/sample_submission.csv")
sample_submission['yield'] = test_preds
sample_submission.to_csv("submission4.csv", index=False)

print("submission4.csv saved.")


In [ ]:
pip install optuna


In [ ]:
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import make_scorer, mean_absolute_error
import numpy as np

X = train[new_features]
y = train['yield']

def objective(trial):
    model = HistGradientBoostingRegressor(
        learning_rate=trial.suggest_float("learning_rate", 0.01, 0.1),
        max_iter=trial.suggest_int("max_iter", 200, 1000),
        max_leaf_nodes=trial.suggest_int("max_leaf_nodes", 15, 63),
        min_samples_leaf=trial.suggest_int("min_samples_leaf", 5, 50),
        l2_regularization=trial.suggest_float("l2_regularization", 0.0, 1.0),
        random_state=42
    )
    score = cross_val_score(model, X, y, cv=3,
                            scoring=make_scorer(mean_absolute_error)).mean()
    return score

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)

print("Best params:", study.best_params)
print("Best MAE:", study.best_value)


# stacking model

In [ ]:


from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import HistGradientBoostingRegressor, RandomForestRegressor, StackingRegressor
from sklearn.linear_model import RidgeCV
from xgboost import XGBRegressor


train = pd.read_csv("/kaggle/input/ml-league-supervised-learning-competition/train.csv")
test = pd.read_csv("/kaggle/input/ml-league-supervised-learning-competition/test.csv")
submission = pd.read_csv("/kaggle/input/ml-league-supervised-learning-competition/sample_submission.csv")


features = ['fruitset', 'seeds', 'fruitmass']
for df in [train, test]:
    df['fruitset_x_mass'] = df['fruitset'] * df['fruitmass']
    df['seeds_div_mass'] = df['seeds'] / (df['fruitmass'] + 1e-6)
    df['fruit_density'] = df['fruitmass'] / (df['seeds'] + 1e-6)

new_features = features + ['fruitset_x_mass', 'seeds_div_mass', 'fruit_density']
X = train[new_features]
y = train['yield']
X_test = test[new_features]


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

base_models = [
    ('hgb', HistGradientBoostingRegressor(random_state=42)),
    ('xgb', XGBRegressor(n_estimators=300, learning_rate=0.05, random_state=42)),
    ('rf', RandomForestRegressor(n_estimators=300, random_state=42))
]

stacked_model = StackingRegressor(
    estimators=base_models,
    final_estimator=RidgeCV(),
    cv=3,
    n_jobs=-1
)


stacked_model.fit(X_train, y_train)


val_preds = stacked_model.predict(X_val)
val_mae = mean_absolute_error(y_val, val_preds)
print(f"📉 Stacked MAE: {val_mae:.4f}")


test_preds = stacked_model.predict(X_test)
submission['yield'] = test_preds
submission.to_csv("submission_stacked.csv", index=False)
print(" submission_stacked.csv saved!")


In [ ]:

from sklearn.ensemble import HistGradientBoostingRegressor



train = pd.read_csv("/kaggle/input/ml-league-supervised-learning-competition/train.csv")
test = pd.read_csv("/kaggle/input/ml-league-supervised-learning-competition/test.csv")
submission = pd.read_csv("/kaggle/input/ml-league-supervised-learning-competition/sample_submission.csv")


for df in [train, test]:
    df['fruitset_x_mass'] = df['fruitset'] * df['fruitmass']
    df['seeds_div_mass'] = df['seeds'] / (df['fruitmass'] + 1e-6)
    df['fruit_density'] = df['fruitmass'] / (df['seeds'] + 1e-6)

features = ['fruitset', 'seeds', 'fruitmass', 'fruitset_x_mass', 'seeds_div_mass', 'fruit_density']
X = train[features]
y = train['yield']
X_test = test[features]


best_model = HistGradientBoostingRegressor(
    learning_rate=0.01096,
    max_iter=607,
    max_leaf_nodes=27,
    min_samples_leaf=6,
    l2_regularization=0.609,
    random_state=42
)
best_model.fit(X, y)


#joblib.dump(best_model, "optuna_best_hgb_model.pkl")


preds = best_model.predict(X_test)
submission['yield'] = preds
submission.to_csv("submission_optuna_final.csv", index=False)

print(" Model and submission saved!")


In [ ]:



train = pd.read_csv("/kaggle/input/ml-league-supervised-learning-competition/train.csv")
test = pd.read_csv("/kaggle/input/ml-league-supervised-learning-competition/test.csv")
submission = pd.read_csv("/kaggle/input/ml-league-supervised-learning-competition/sample_submission.csv")

for df in [train, test]:
    df['fruitset_x_mass'] = df['fruitset'] * df['fruitmass']
    df['fruitset_x_seeds'] = df['fruitset'] * df['seeds']
    df['mass_x_seeds'] = df['fruitmass'] * df['seeds']
    df['combo_feature'] = df['fruitset'] * df['fruitmass'] * df['seeds']

features = ['fruitset', 'fruitmass', 'seeds', 'fruitset_x_mass', 'fruitset_x_seeds', 'mass_x_seeds', 'combo_feature']
X = train[features]
y = train['yield']
X_test = test[features]


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model = HistGradientBoostingRegressor(
    learning_rate=0.01096,
    max_iter=607,
    max_leaf_nodes=27,
    min_samples_leaf=6,
    l2_regularization=0.609,
    random_state=42
)
model.fit(X_train, y_train)
val_mae = mean_absolute_error(y_val, model.predict(X_val))
print(f"📉 Validation MAE: {val_mae:.4f}")


model.fit(X, y)
preds = model.predict(X_test)

#joblib.dump(model, "final_engineered_model.pkl")


submission['yield'] = preds
submission.to_csv("submission_final_engineered.csv", index=False)
print("submission_6.csv and model saved.")


In [ ]:





drop_cols = ['clonesize', 'Row#', 'RainingDays', 'AverageRainingDays', 'honeybee', 'andrena', 'bumbles']
train = train.drop(columns=drop_cols, errors='ignore')
test = test.drop(columns=drop_cols, errors='ignore')


for df in [train, test]:
    df['fruitset_x_seeds'] = df['fruitset'] * df['seeds']
    df['fruitmass_x_seeds'] = df['fruitmass'] * df['seeds']
    df['combo_feature'] = df['fruitset'] * df['fruitmass'] * df['seeds']
    df['fruit_density'] = df['fruitmass'] / (df['seeds'] + 1e-6)
    df['osmia_boost'] = df['fruitset'] * df['osmia']


features = [
    'fruitset', 'fruitmass', 'seeds', 'osmia',
    'fruitset_x_seeds', 'fruitmass_x_seeds',
    'combo_feature', 'fruit_density', 'osmia_boost'
]


X = train[features]
y = train['yield']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


model = HistGradientBoostingRegressor(
    learning_rate=0.01,
    max_iter=650,
    max_leaf_nodes=27,
    min_samples_leaf=6,
    l2_regularization=0.6,
    random_state=42
)


model.fit(X_train, y_train)
val_preds = model.predict(X_val)
val_mae = mean_absolute_error(y_val, val_preds)
print(f" Validation MAE: {val_mae:.4f}")


model.fit(X, y)
test_preds = model.predict(test[features])

# Save outputs
submission['yield'] = test_preds
submission.to_csv("submission_target_mae230.csv", index=False)
joblib.dump(model, "model_target_mae230.pkl")
print(" submission_target_mae230.csv and model saved.")


In [ ]:
import numpy as np


drop_cols = ['clonesize', 'Row#', 'RainingDays', 'AverageRainingDays', 'honeybee', 'andrena', 'bumbles']
train.drop(columns=drop_cols, errors='ignore', inplace=True)
test.drop(columns=drop_cols, errors='ignore', inplace=True)


for col in ['fruitset', 'fruitmass', 'seeds', 'yield']:
    q_low = train[col].quantile(0.01)
    q_high = train[col].quantile(0.99)
    train[col] = train[col].clip(lower=q_low, upper=q_high)


for df in [train, test]:
    df['fruitset_x_seeds'] = df['fruitset'] * df['seeds']
    df['fruitmass_x_seeds'] = df['fruitmass'] * df['seeds']
    df['combo_feature'] = df['fruitset'] * df['fruitmass'] * df['seeds']
    df['fruit_density'] = df['fruitmass'] / (df['seeds'] + 1e-6)
    df['osmia_boost'] = df['fruitset'] * df['osmia']
    df['mass_to_set'] = df['fruitmass'] / (df['fruitset'] + 1e-6)
    df['log_seeds'] = np.log1p(df['seeds'])


features = [
    'fruitset', 'fruitmass', 'seeds', 'osmia',
    'fruitset_x_seeds', 'fruitmass_x_seeds',
    'combo_feature', 'fruit_density', 'osmia_boost',
    'mass_to_set', 'log_seeds'
]

X = train[features]
y = train['yield']
X_test = test[features]


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


model = HistGradientBoostingRegressor(
    learning_rate=0.008,
    max_iter=900,
    max_leaf_nodes=31,
    min_samples_leaf=5,
    l2_regularization=0.4,
    random_state=42
)


model.fit(X_train, y_train)
val_preds = model.predict(X_val)
val_mae = mean_absolute_error(y_val, val_preds)
print(f" FINAL MAE: {val_mae:.4f}")

model.fit(X, y)
submission['yield'] = model.predict(X_test)
submission.to_csv("submission_ultra_tuned.csv", index=False)

print("submission and model saved.")


In [ ]:



features = ['fruitset', 'fruitmass', 'seeds', 'osmia']
X = train[features]
y = train['yield']
X_test = test[features]


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


model = HistGradientBoostingRegressor(
    learning_rate=0.03,
    max_iter=600,
    max_leaf_nodes=31,
    random_state=42
)

model.fit(X_train, y_train)
val_preds = model.predict(X_val)
val_mae = mean_absolute_error(y_val, val_preds)
print(f"🧪 MAE (Simple Model): {val_mae:.2f}")

# Retrain and predict
model.fit(X, y)
submission['yield'] = model.predict(X_test)
#submission.to_csv("submission_simplified.csv", index=False)

print("✅ Submission and model saved.")


In [ ]:

from sklearn.model_selection import KFold, cross_val_predict


train['log_yield'] = np.log1p(train['yield'])


def engineer_features(df):
    df['fruitset_x_seeds'] = df['fruitset'] * df['seeds']
    df['fruitmass_x_seeds'] = df['fruitmass'] * df['seeds']
    df['combo_feature'] = df['fruitset'] * df['fruitmass'] * df['seeds']
    df['fruit_density'] = df['fruitmass'] / (df['seeds'] + 1e-6)
    df['osmia_boost'] = df['fruitset'] * df['osmia']
    df['bee_sum'] = df['honeybee'] + df['bumbles'] + df['andrena'] + df['osmia']
    df['raining_index'] = df['RainingDays'] + df['AverageRainingDays']
    return df

train = engineer_features(train)
test = engineer_features(test)

# Final feature list
features = [
    'fruitset', 'fruitmass', 'seeds', 'osmia', 'honeybee', 'bumbles', 'andrena',
    'RainingDays', 'AverageRainingDays',
    'fruitset_x_seeds', 'fruitmass_x_seeds', 'combo_feature',
    'fruit_density', 'osmia_boost', 'bee_sum', 'raining_index'
]

X = train[features]
y_log = train['log_yield']
X_test = test[features]

# 5-Fold Cross Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
model = HistGradientBoostingRegressor(
    learning_rate=0.03,
    max_iter=600,
    max_leaf_nodes=31,
    min_samples_leaf=5,
    random_state=42
)

# Out-of-fold predictions
oof_preds_log = cross_val_predict(model, X, y_log, cv=kf)
oof_preds = np.expm1(oof_preds_log)
true_y = train['yield']
cv_mae = mean_absolute_error(true_y, oof_preds)
print(f" Cross-validated MAE: {cv_mae:.2f}")

# Final training and submission
model.fit(X, y_log)
test_preds = np.expm1(model.predict(X_test))
submission['yield'] = test_preds
submission.to_csv("submission_generalized_cv.csv", index=False)
#joblib.dump(model, "model_generalized_cv.pkl")
print(" Model and submission saved!")


In [ ]:

train = pd.read_csv("/kaggle/input/ml-league-supervised-learning-competition/train.csv")
test = pd.read_csv("/kaggle/input/ml-league-supervised-learning-competition/test.csv")
submission = pd.read_csv("/kaggle/input/ml-league-supervised-learning-competition/sample_submission.csv")


train['log_yield'] = np.log1p(train['yield'])

features = ['fruitset','seeds']

X = train[features]
y_log = train['log_yield']
X_test = test[features]


kf = KFold(n_splits=5, shuffle=True, random_state=42)
model = HistGradientBoostingRegressor(
    learning_rate=0.03,
    max_iter=600,
    max_leaf_nodes=31,
    min_samples_leaf=5,
    random_state=42
)


oof_preds_log = cross_val_predict(model, X, y_log, cv=kf)
oof_preds = np.expm1(oof_preds_log)
cv_mae = mean_absolute_error(train['yield'], oof_preds)
print(f"📉 Cross-Validated MAE: {cv_mae:.2f}")


model.fit(X, y_log) 
perm_importance = permutation_importance(model, X, y_log, n_repeats=5, random_state=42)
importances = perm_importance.importances_mean

print("\n🔍 Permutation Feature Importance:")
for feat, imp in sorted(zip(features, importances), key=lambda x: x[1], reverse=True):
    print(f"  {feat}: {imp:.4f}")


print(f"\n FINAL CV MAE (Log-Transformed Model): {cv_mae:.2f}")
test_preds_log = model.predict(X_test)
test_preds = np.expm1(test_preds_log)
submission['yield'] = test_preds
submission.to_csv("submission8.csv", index=False)

print("\nModel and submission saved.")


In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.metrics import mean_absolute_error


train = pd.read_csv("/kaggle/input/ml-league-supervised-learning-competition/train.csv")
test = pd.read_csv("/kaggle/input/ml-league-supervised-learning-competition/test.csv")
submission = pd.read_csv("/kaggle/input/ml-league-supervised-learning-competition/sample_submission.csv")


y = train['yield']
features = ['fruitset','seeds']
X = train[features]
X_test = test[features]


model = XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

kf = KFold(n_splits=5, shuffle=True, random_state=42)
oof_preds = cross_val_predict(model, X, y, cv=kf)
cv_mae = mean_absolute_error(y, oof_preds)
print(f" CV MAE (XGBoost Generalized): {cv_mae:.2f}")

model.fit(X, y)
submission['yield'] = model.predict(X_test)
submission.to_csv("submission_boosted_generalized.csv", index=False)
